In [52]:
# Import all necessary libraries
from sklearn.datasets import make_moons
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier# Step1: Create data set
from sklearn import tree
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.tree import plot_tree
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, roc_curve, auc, classification_report
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import GridSearchCV
from scipy.special import expit
from sklearn.metrics import classification_report
from sklearn import datasets  
from sklearn.metrics import accuracy_score 
from IPython.display import Image  
from sklearn.externals.six import StringIO  
from sklearn.model_selection import ShuffleSplit
from sklearn.base import clone
from scipy.stats import mode

In [53]:
# Define the number of examples.
N = 1000
# gerar conjunto de Luas 
x,y =  make_moons(n_samples=10000, noise=0.4, random_state=42)

In [54]:
#treinamento do conjunto 
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [55]:
# Use o Grid Search com validação cruzada
#(tente varios valores para max_leaf_node)

params = param_grid = [{'max_leaf_nodes': [None, 2, 3, 4, 5, 10, 15, 20, 30, 50, 60, 70, 90]}]
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1, verbose=1, cv=3)
grid_search_cv.fit(X_train, y_train)

Fitting 3 folds for each of 13 candidates, totalling 39 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  39 out of  39 | elapsed:    0.2s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid=[{'max_leaf_nodes': [None, 2, 3, 4, 5, 10, 15, 20, 30,
   

In [56]:
n_trees = 1000
n_instances = 100

mini_sets = []
mini_sets2=[]

rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=42)
#USAR O PARAMETRO train_size TAMBEM SERVE E DA O RESULTADO IDENTICO 

rs2 = ShuffleSplit(n_splits=n_trees, train_size=n_instances,random_state=42) #tambem serve

for mini_train_index, mini_test_index in rs.split(X_train):
    #print("TRAIN size:", len(mini_train_index), "TEST size:", len(mini_test_index) )
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

for mini_train_index, mini_test_index in rs2.split(X_train):
    #print("TRAIN size:", len(mini_train_index), "TEST size:", len(mini_test_index) )
    X_mini_train2 = X_train[mini_train_index]
    y_mini_train2 = y_train[mini_train_index]
    mini_sets2.append((X_mini_train2, y_mini_train2))
# As "n_trees" amostras de treinamento estão em "mini_sets".
print(len(mini_sets))
print(len(mini_sets2))
print(mini_sets[0][0][0],mini_sets2[0][0][0])
#são identicos

1000
1000
[0.9645406  1.00563146] [0.9645406  1.00563146]


In [57]:
# copiando a melhor configuração da tree para todas as "n_trees" trees. 
forest = [clone(grid_search_cv.best_estimator_) for _ in range(n_trees)]

accuracy_scores = []
for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    # para cada conjunto de treinamento (mini_train), treinamos as nossas trees
    tree.fit(X_mini_train, y_mini_train)
    # fazemos a predição no conjunto de teste
    y_pred = tree.predict(X_test)
    # e calculamos a acurácia de cada predição
    accuracy_scores.append(accuracy_score(y_test, y_pred))
# média das acurácias
np.mean(accuracy_scores)

0.7961612

In [58]:
print(np.max(accuracy_scores),np.min(accuracy_scores))

0.85 0.6972
